In [ ]:
from os import getcwd, chdir, path, listdir
import warnings
import pickle
%load_ext autoreload
from importlib import reload
import time
warnings.filterwarnings("ignore")

chdir(
    path.join(
        path.expanduser('~'), 
        'PycharmProjects', 
        'DiplomaThesis'
    )
)
print(getcwd())

In [ ]:
import sql_handler
import User
import Category
import Deal
import Event
import Partner
import config
import vector
import lstm

reload(lstm)
reload(Category)
reload(Event)
reload(Deal)
reload(Partner)
reload(sql_handler)
reload(User)
reload(config)
reload(vector)

Count outlier limits ['low', 'high'] for user event amounts

According to the formula:
	Lower limit = Fixed to minumum cfg.user_event_amount_minimum
	Upper limit = Q3 + 1.5*IQR (general outlier formula, counted from cookies records with > cfg.user_event_amount_minimum) 

In [ ]:
User.count_and_save_outlier_borders()

Create and save Deals
	-save list of unique deal_id(s) from user events(targeting deals)
	-save deal objects as dictionary hashing under deal_id for fast lookup
 	

In [ ]:
Deal.save_all_used_deal_ids()
print(len(Deal.load_all_used_deal_ids()))

Deal.create_and_save_all_deals()

Create and save Categories
		
	-save list of unique category_id(s)
	-save category objects as dictionary hashing under category_id
 	

In [ ]:
Category.Category.create_and_save_categories()

Define event type medians
	Each event type has counted median event duration value in seconds from defined amount of cookies.
	Values are used to substitute from missing values in last session event (>1800s) and last user event in records.
	Event type medians are saved into file defined in config as a dictionary hashing durations under event type names.

	This takes a LONG time and you have probably been provided with the ".dict" file already.

In [ ]:
categories = Category.Category.load_categories()
deals = Deal.load_all_deals()

User.count_and_save_event_medians(cookie_limit=1000000, categories=categories, deals=deals)
User.load_event_medians(debug=True)

Create and save Partners
	
	-save list of unique partner_id(s)
	-save partner objects as dictionary hashing under partner_id

In [ ]:
Partner.create_and_save_partners()

Create and save User objects
	users are saved in a batches of amount(defined in config.pickle_wrap_amount) in 'users' dir

In [ ]:
raw_user_creation_start = time.time()
User.create_and_save_all_users()
raw_user_creation_end = time.time()
print('Total time for creating users: {}'.format(
    str(raw_user_creation_end-raw_user_creation_start))
)

Save a dictionary of normalization values for specific features.
	Outlier amounts are fetched on the Postgres database side by counting 1st and 3rd quartile.
	Normalization of features is according to the formula:
	
	(float(value) - min_value) / (max_value - min_value)
	
	where:
		value = value to normalize
		min_value = bottom outlier value
		max_value = top outlier value
	
	outlier values are counted according to the general outlier formula:
	
		Lower limit = Q1 - 1.5*IQR
		Upper limit = Q3 + 1.5*IQR
	
	*sometimes upper or lower limit is overridden by case-specific value
	(for example upper limit on rating of partner is 5.0 as fixed by amount of stars possible)
	
	

In [ ]:
vector.save_normalization_values()

Pre-fetch partners for input and target vectors

	Total dataset partners: ~7000
	Select only partners in filtered user events to make vectors more sparse

	Result:
		~2000 for target vectors
		~3600 for input vectors
	
	Inputs contain wider variety of partners because there is ~timesteps*amount_of_users events for partners to be in
	Target vectors only contain ~amount_of_users amount of events, thus less unique partner IDs

In [ ]:
deals = Deal.load_all_deals()
partners = Partner.load_all_partners()
categories = Category.Category.load_categories()

users = []
for filename in listdir(path.join(getcwd(), config.dir_users)):
    
    with open(path.join(getcwd(), config.dir_users, filename), 'rb') as user_file:
        users += pickle.load(user_file)

input_partners, users_with_found_input_partners = User.get_input_partner_ids(users, deals, partners)
target_partners, users_with_found_target_partners = User.get_target_partner_ids(users, deals, partners)


print(len(target_partners))
print(len(input_partners))
print(users_with_found_input_partners)
print(users_with_found_target_partners)

Create vectors and store them by the config.pickle_wrap_amount
	If less than pickle is stored(95%-100%) it is because some users had no event with targetable partner in the second split part of events
	Dataset should retain 98-99% of users in form of vectors with 10 target-source events. 
	More target-source events per user --> lower percentual dropout

In [ ]:
n_vectors_created = 0

for filename in listdir(path.join(getcwd(), config.dir_users)):
    print(path.join(getcwd(), config.dir_users, filename))
    n_vectors_created += vector.create_userfile_vectors(
        filename, 
        deals, 
        partners, 
        categories, 
        input_partners, 
        target_partners
    )

print(f'Created {n_vectors_created} user vectors from users')

Sort vectors into separate directory by target partner(class)
	directory differs based on using or not using partner one-hot-subvector on input vectors

In [ ]:
vector.sort_and_save_vectors_by_target_partners(target_partners)